# Ex - 5
## BigGIS for BigDATA (Quantum Computing)
### Ashutosh Kumar Jha (Scientist/Engineer SF), IIRS, ISRO
### Sudikin Pramanik (Quantum computing coordinator), Student, IIRS, ISRO
`Quantum Machine learning`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install qiskit qiskit_aer qiskit_ibm_runtime qiskit_machine_learning gdal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69

Loading important libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from qiskit_aer import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [ ]:
!conda list

/bin/bash: line 1: conda: command not found


In [ ]:
import numpy as np
import pandas as pd
from osgeo import gdal
import matplotlib.pyplot as plt

# Function to read a TIFF image and return its bands
def read_tiff_image(image_path):
    # Open the TIFF file
    dataset = gdal.Open(image_path)

    # Get the number of bands
    num_bands = dataset.RasterCount

    # Read each band into a numpy array
    bands = []
    for i in range(1, num_bands + 1):
        band = dataset.GetRasterBand(i).ReadAsArray()
        bands.append(band)

    return np.array(bands)

# Function to read label TIFF (0 and 1 labels)
def read_label_tiff(label_path):
    dataset = gdal.Open(label_path)
    label = dataset.GetRasterBand(1).ReadAsArray()
    return label

# Path to your TIFF image and label
image_path = '/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/dehra_multi.tiff'
label_path = '/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/dehra_label.tiff'


# Step 1: Read the multi-band image
bands = read_tiff_image(image_path)

# Step 2: Read the label image (binary labels 0 and 1)
labels = read_label_tiff(label_path)

# Step 3: Check if image and label dimensions match
if bands.shape[1:] != labels.shape:
    raise ValueError(f"Dimensions do not match! Image size: {bands.shape[1:]}, Label size: {labels.shape}")

# Step 4: Flatten each band and the label image
flattened_bands = [band.flatten() for band in bands]
flattened_labels = labels.flatten()

# Step 5: Combine bands and labels into a DataFrame
data = np.column_stack(flattened_bands + [flattened_labels])
band_names = [f'band{i+1}' for i in range(len(bands))]
column_names = band_names + ['label']
df = pd.DataFrame(data, columns=column_names)

# Step 6: Convert label column to integers (ensures labels are integers)
df['label'] = df['label'].astype(int)

# Step 7: Show the DataFrame (first few rows)
print(df.head())

# Optional: Save DataFrame to CSV with label column as integer
df.to_csv('/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/output_with_int_labels.csv', index=False)


   band1   band2   band3    band4  label
0  78.25  294.75  313.75  1804.00      1
1  86.00  312.00  310.75  1895.75      1
2  65.00  327.00  278.50  2167.00      1
3  64.00  281.75  262.75  1979.50      1
4  34.25  216.75  194.50  1671.00      1


Loading the datasets and viewing the elements of the data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/output_with_int_labels.csv')
data.head()

,band1,band2,band3,band4,label
0,78.25,294.75,313.75,1804.00,1
1,86.00,312.00,310.75,1895.75,1
2,65.00,327.00,278.50,2167.00,1
3,64.00,281.75,262.75,1979.50,1
4,34.25,216.75,194.50,1671.00,1


Using describe() function to view the statistical data

In [ ]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
band1,693.0,457.586219,373.523589,3.25,174.25,291.50,717.00,1803.50
band2,693.0,752.625902,326.133184,163.50,528.25,675.25,959.50,1806.00
band3,693.0,738.800866,450.272515,144.25,405.50,557.50,1032.00,2438.00
band4,693.0,2313.484127,421.334370,1090.75,1979.50,2350.75,2613.75,3477.25
label,693.0,0.460317,0.498783,0.00,0.00,0.00,1.00,1.00


Separating the features and labels

In [ ]:
X = data.iloc[:,:-1].values
y = data['label'].values

In [ ]:
from scipy.stats import describe
print(describe(y))

DescribeResult(nobs=693, minmax=(0, 1), mean=0.4603174603174603, variance=0.2487842921368933, skewness=0.15923243882462076, kurtosis=-1.9746450304259633)


# Normalize the data

The **`StandardScaler`** in scikit-learn is used to standardize features by removing the mean and scaling to unit variance. It is a preprocessing step that ensures features are on the same scale, which is particularly important for many machine learning algorithms that are sensitive to the magnitude of input features (e.g., SVM, logistic regression, k-means, PCA).

### What It Does:
1. **Removes the Mean**:
   It centers the data by subtracting the mean of each feature.
   $$
   x' = x - \text{mean}
   $$

2. **Scales to Unit Variance**:
   It divides each feature by its standard deviation.
   $$
   x'' = \frac{x'}{\text{std}}
   $$

   After scaling:
   - The mean of each feature will be **0**.
   - The standard deviation of each feature will be **1**.

### Why Standardize?
- Ensures that all features contribute equally to the model.
- Avoids bias toward features with larger scales.
- Accelerates convergence in optimization algorithms.



In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Viewing the stastics of x

In [ ]:
X_df = pd.DataFrame(X)
print(X_df)

            0         1         2         3
0   -1.016295 -1.404968 -0.944668 -1.210089
1   -0.995532 -1.352037 -0.951335 -0.992171
2   -1.051794 -1.306010 -1.023010 -0.347918
3   -1.054473 -1.444858 -1.058014 -0.793254
4   -1.134177 -1.644307 -1.209699 -1.525981
..        ...       ...       ...       ...
688  1.332642  1.421839  1.309484 -0.972576
689  1.417034  1.305238  1.331709 -1.225527
690  1.340009  1.515426  1.681750 -0.277852
691  1.822924  1.918927  1.677305  0.718513
692  0.374849  0.506674  0.339371  0.781454

[693 rows x 4 columns]


In [ ]:
X_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,693.0,0.000000e+00,1.000722,-1.217231,-0.759098,-0.444968,0.695006,3.605893
1,693.0,1.640503e-16,1.000722,-1.807701,-0.688485,-0.237424,0.634782,3.232222
2,693.0,-1.230377e-16,1.000722,-1.321378,-0.740755,-0.402938,0.651630,3.776438
3,693.0,4.921508e-16,1.000722,-2.904148,-0.793254,0.088511,0.713169,2.764091


Using PCA to reduce the dimensionality of the data. Now we use PCA as it is essential to reduce the data set size for proper processing in Quantum machine learning.

We can see that when it has been converted to PCA the original values of the columns are also changed. It has created its own values in the process. This means that the PCA is more important as a tool of dimensionality reduction of the number of features.

In [ ]:
n_component =4

In [ ]:
pca = PCA(n_component)
X_pca = pca.fit_transform(X)
print(describe(X_pca))
XPca_df = pd.DataFrame(X_pca)
print(XPca_df)
XPca_df.describe().transpose()
X_train, X_test, y_train, y_test = train_test_split(X_pca,y, test_size=0.1, random_state = 2)

DescribeResult(nobs=693, minmax=(array([-2.36875033, -2.98716762, -1.23696497, -0.30932191]), array([5.44468268, 2.77411144, 3.31274318, 0.75867607])), mean=array([ 2.08106740e-16,  2.74111341e-16,  6.85678867e-17, -8.88639010e-18]), variance=array([2.91221464, 1.01447973, 0.06391821, 0.01516777]), skewness=array([ 0.93796836, -0.1921729 ,  4.67260925,  1.28408644]), kurtosis=array([-4.50422782e-02, -5.31019091e-01,  5.57446898e+01,  4.28845033e+00]))
            0         1         2         3
0   -1.863768 -1.358924  0.191303  0.081238
1   -1.838686 -1.137397  0.160373  0.091497
2   -1.926100 -0.492715  0.118545  0.113346
3   -1.999798 -0.953122  0.140856  0.143868
4   -2.202080 -1.704172  0.115598  0.123965
..        ...       ...       ...       ...
688  2.402166 -0.804989 -0.072427 -0.185896
689  2.412656 -1.075927 -0.040361 -0.082695
690  2.630528 -0.097899  0.205629 -0.161501
691  3.078161  0.916579 -0.128821  0.056181
692  0.654759  0.830408 -0.064383  0.025326

[693 rows x 4 c

Train Test split

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 2)

In [ ]:
print(describe(X_train))
print(describe(X_test))

DescribeResult(nobs=623, minmax=(array([-2.36875033, -2.98716762, -1.23696497, -0.30932191]), array([5.44468268, 2.77411144, 3.31274318, 0.75867607])), mean=array([0.03387337, 0.00612019, 0.00012613, 0.00187271]), variance=array([2.94129864, 1.02916807, 0.0635941 , 0.01514321]), skewness=array([ 0.89629689, -0.19324597,  5.06040974,  1.243633  ]), kurtosis=array([-0.12607475, -0.53488738, 60.54861789,  4.22551857]))
DescribeResult(nobs=70, minmax=(array([-2.1020702 , -2.43985982, -1.08262493, -0.19558401]), array([4.67616704, 1.83917836, 1.47198626, 0.56040864])), mean=array([-0.30147296, -0.05446971, -0.00112259, -0.01666713]), variance=array([2.58968008, 0.89342631, 0.06776475, 0.01529545]), skewness=array([ 1.36966634, -0.20842466,  1.47099175,  1.68650524]), kurtosis=array([ 1.07203477, -0.53930982, 16.93259583,  5.26572154]))


Creating the Quantum Feature map and the Quantum kernel

In [ ]:
feature_map = ZZFeatureMap(feature_dimension = 4, entanglement = 'full', reps=2)
# Please use the other feature maps as desired like Pauli, Zfeaturemap, 1local and 2local.
quantum_kernel = FidelityQuantumKernel(feature_map = feature_map)

Training the QSVM classifier

In [ ]:
classifier = QSVC(quantum_kernel = quantum_kernel)
classifier.fit(X_train, y_train)

In [ ]:
import joblib

# Save the trained classifier
joblib.dump(classifier, '/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/trained_qsvc_model2.pkl')


In [ ]:
y_pred = classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Creating a new function to predict whether a person has Forest or not

In [ ]:
def predict_label(new_data):
    # Normalizing the new data
    new_data_normalized = scaler.transform(new_data)

    # Make prediction
    prediction = classifier.predict(new_data_normalized)

    #Map prediction to label
    return 'Positive' if prediction[0]==1 else 'Negative'

When ever we are entering the new data we are entering the entire data. But if you remember about PCA in which we took only 4 features, it implies that what ever the number of features we take we are about to get the right amount of result out of it when we input data with all the features.

In [ ]:
#Example new data (replace with actual data)
new_data = np.array([[86,300,300,1200]])

#Predict and print the result
print(predict_label(new_data))

In [ ]:
import numpy as np
from osgeo import gdal
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from tqdm.notebook import tqdm  # For progress bar in notebooks

# Function to read a multi-band TIFF image
def read_tiff_image(image_path):
    dataset = gdal.Open(image_path)
    if dataset is None:
        raise FileNotFoundError(f"Failed to open the file: {image_path}")

    num_bands = dataset.RasterCount
    bands = []
    for i in range(1, num_bands + 1):
        band = dataset.GetRasterBand(i).ReadAsArray()
        bands.append(band)

    return np.array(bands), dataset

# Function to predict the label for a given pixel (feature vector)
def predict_pixel(features):
    # Normalize and reduce using the trained scaler and PCA
    normalized_features = scaler.transform([features])

    # Make prediction
    prediction = classifier.predict(normalized_features)

    return prediction[0]  # Return 1 for Positive, 0 for Negative

# Function to generate a georeferenced TIFF image with predicted labels
def save_predictions_as_geotiff(predictions, output_path, reference_dataset):
    # Get image dimensions
    height, width = predictions.shape

    # Create a new georeferenced TIFF file with the same spatial reference as the input
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_path, width, height, 1, gdal.GDT_Byte)
    output_dataset.SetGeoTransform(reference_dataset.GetGeoTransform())
    output_dataset.SetProjection(reference_dataset.GetProjection())

    # Write the predictions to the new file
    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(predictions)

    # Flush cache and close the dataset
    output_band.FlushCache()
    output_dataset = None

# Path to the input multi-band TIFF image
input_image_path = r'/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/test1.tiff'
output_image_path = r'/content/drive/MyDrive/Sudikin tehri 3d/BigAI_Quantum_Day1/Level 2/olabel1.tiff'

# Step 1: Read the input multi-band TIFF image
bands, dataset = read_tiff_image(input_image_path)

# Step 2: Initialize the prediction array
height, width = bands.shape[1], bands.shape[2]
predictions = np.zeros((height, width), dtype=np.uint8)

# Step 3: For each pixel, generate the feature vector and predict
# Using tqdm.notebook to track progress in the notebook
for i in tqdm(range(height), desc="Processing rows", leave=False):
    for j in tqdm(range(width), desc="Processing pixels", leave=False):
        # Extract the feature vector for the pixel (using all bands)
        pixel_features = [bands[band_idx, i, j] for band_idx in range(bands.shape[0])]

        # Predict the label (0 or 1)
        predictions[i, j] = predict_pixel(pixel_features)

# Step 4: Save the predictions as a new georeferenced TIFF image
save_predictions_as_geotiff(predictions, output_image_path, dataset)

print(f"Predicted label image saved to {output_image_path}")
